In [1]:
import os

In [ ]:
import os
print(os.getcwd())

'c:\\Users\\hp\\Desktop\\MLOps project\\End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC\\research'

In [3]:
os.chdir("../")

In [ ]:
import os
print(os.getcwd())

'c:\\Users\\hp\\Desktop\\MLOps project\\End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    min_samples_split: int
    min_samples_leaf: int
    bootstrap: bool
    target_column: str

In [6]:
import pandas as pd
from box import ConfigBox
from src.mlops_project.logger import logging
from src.mlops_project.constants import *
from src.mlops_project.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = ConfigBox(read_yaml(config_filepath))
        self.schema = ConfigBox(read_yaml(schema_filepath))
        self.params = ConfigBox(read_yaml(params_filepath))
        
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            min_samples_split=params.min_samples_split,
            min_samples_leaf=params.min_samples_leaf,
            bootstrap=params.bootstrap,
            target_column=schema.name
        )

        return model_trainer_config

In [9]:
from sklearn.ensemble import RandomForestClassifier
import joblib

In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        try:
            logging.info("Reading train and test data")
            train_data = pd.read_csv(self.config.train_data_path)
            test_data = pd.read_csv(self.config.test_data_path)

            logging.info("Splitting the dataset into features and target")
            X_train = train_data.drop([self.config.target_column], axis=1)
            X_test = test_data.drop([self.config.target_column], axis=1)
            y_train = train_data[[self.config.target_column]]
            y_test = test_data[[self.config.target_column]]

            logging.info("Training the model")
            rfc = RandomForestClassifier(
                n_estimators=self.config.n_estimators,
                max_depth=self.config.max_depth,
                min_samples_split=self.config.min_samples_split,
                min_samples_leaf=self.config.min_samples_leaf,
                bootstrap=self.config.bootstrap
            )

            rfc.fit(X_train, y_train)

            logging.info("Saving the model")
            joblib.dump(rfc, os.path.join(self.config.root_dir, self.config.model_name))

        except Exception as e:
            raise e

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e


[2025-02-05 00:10:58,702: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-05 00:10:58,713: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-05 00:10:58,721: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-05 00:10:58,726: INFO: common: created directory at: artifacts]
[2025-02-05 00:10:58,729: INFO: common: created directory at: artifacts/model_trainer]
[2025-02-05 00:10:58,732: INFO: 3967477003: Reading train and test data]
[2025-02-05 00:10:58,861: INFO: 3967477003: Splitting the dataset into features and target]
[2025-02-05 00:10:58,870: INFO: 3967477003: Training the model]


c:\Users\hp\anaconda3\envs\customer_churn_project\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[2025-02-05 00:11:01,261: INFO: 3967477003: Saving the model]
